# 利用AdaBoost元算法提高分类性能

## AdaBoost（adaptive boosting）是自适应boosting的缩写。 
**其运行过程为：**

**训练数据中的每个样本，并赋予其一个权重，这些权重构成了向量D。**

**首先在训练数据上训练出一个弱分类器并计算该分类器的错误率，然后再在同一数据集上再次训练弱分类器。**

**在第二次训练分类器时，重新调整样本权重，预测错误的样本权重将会增加。**

**同时，每个分类器都分配了一个alpha权重值，以此来得到所有分类器的最终分类结果。**

其中，错误率 
$$\begin{aligned}
\epsilon = \frac{错误分类结果数}{所有样本数}
\end{aligned}$$
而 
$$\begin{aligned}
\alpha = \frac {1}{2} ln(\frac {1 - \epsilon}{\epsilon})
\end{aligned}$$

计算出alpha值之后，我们对D进行更新，其计算如下。

如果分类正确：
$$\begin{aligned}
D_i^{(t+1)} = \frac {D_i^{(t)}e^{-\alpha}}{Sum(D)}
\end{aligned}$$
如果分类错误：
$$\begin{aligned}
D_i^{(t+1)} = \frac {D_i^{(t)}e^{\alpha}}{Sum(D)}
\end{aligned}$$

## 基于单层决策树构建弱分类器